# <center> TensorFlow </center>

![](tflogo.png)

*<p style='text-align:right'>By TensorFlow - vectors combined, edited - Begoon [Apache License 2.0] via Wikimedia Commons</p>*

- one of the most popular tools for deep learning
- released in 2015
- "framework for numerical computation based on dataflow graphs." [2] [Hope et al. (2017)]
- tensors - multidimensional arrays
- built in C++, used through higer level languages (Python) and libraries (keras, tf-slim)

![](sqcube.png)

![](greyrgb.png)

- image tensor flows through the computational graph

### Computational graph:

![](compgraph.png)

- composed of nodes and edges

*<p style='text-align:right'>Computational Graph. By Sebastian Raschka via Github</p>*

### Advantages and Disadvantages (a few):

- open source and continously developed
- good visualization tools 
- scalability and portability
- steeper learning curve to become good at it 
- implies solid knowledge of math
- http://tensorflow.org

### Types of Operations:

- Add, Sub, Div, Mul, Log, Exp, etc. (element wise math ops.)
- on arrays and matrices: Constant, Shape, Split, Concat, Slice, MatMul, etc.
- Sigmoid, ReLU, Convolution2D, SoftMax, etc.
- and more

In [22]:
import tensorflow as tf
import numpy as np

In [23]:
a = tf.constant('Hello')
b = tf.constant(' World!')
c = a+b

In [24]:
print(c)

Tensor("add_3:0", shape=(), dtype=string)


In [25]:
with tf.Session() as sess: 
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [26]:
c = tf.add(a,b)

with tf.Session() as sess:
    runop = sess.run(c)
    
print(runop)

b'Hello World!'


In [27]:
mat_a = tf.constant(np.arange(1,12, dtype=np.int32), shape=[2,2,3])
mat_b = tf.constant(np.arange(12,24, dtype=np.int32), shape=[2,3,2])
mul_c = tf.matmul(mat_a, mat_b)

with tf.Session() as sess:
    runop = sess.run(mul_c)
    
print(runop)

[[[ 88  94]
  [214 229]]

 [[484 508]
  [642 674]]]


### A typical TensorFlow project:

- (1). constructing the graph: tf.constant, tf.placeholder, tf.variable, tf.nn.sigmoid, tf.matmul, etc.
- (2). executing the graph (within a session)
- use tensorboard for visualization.

### Placeholders:

- allow for reusability
- can be populated with different data every time you run the graph (feed_dict)

In [28]:
#with tf.Session() as sess:
#    sess.run(ops)
#    sess.close()

In [29]:
x = tf.placeholder(tf.float32, name='X', shape=(4,9))
w = tf.placeholder(tf.float32, name='W', shape=(9,1))
b = tf.fill((4,1), -1., name='bias')
y = tf.matmul(x,w)+b
s = tf.reduce_max(y)

In [30]:
x_data = np.random.randn(4,9)
w_data = np.random.randn(9,1)

with tf.Session() as sess:
    out_p = sess.run(s, feed_dict={x:x_data, w:w_data})
    
print('Output: {}'.format(out_p))

Output: 0.8661351203918457


### Optimization

- purpose of a typical project: learn associations between features and labels.

### <center> $f(x_{i}) = w^{T}x_{i} + b$ </center> 
### <center> $y_{i} = f(x_{i}) + \varepsilon_{i}$ </center>

- **Loss Function**: how far the model is from the truth.
- train the model to minimize loss (adjusting the parameters).
- **MSE (Mean Squared Error)** - averages square distances (y_true, y_pred):

## <center> $L(y, \hat y) = \frac{1}{n}\ ∑_{i=1}^{n}(y_{i}-\hat y_{i})^{2}$ </center>

- **Cross Entropy** - similarity between distributions:

## <center> $H(p,q)=-∑_{x}p(x)logq(x)$ </center>

In [31]:
# X, Y, W, b

x = tf.placeholder(tf.float32, shape=[None, 3])
y_true = tf.placeholder(tf.float32, shape=None)
w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
b = tf.Variable(0, dtype=tf.float32, name='b')

# Output

y_pred = tf.matmul(w, tf.transpose(x))+b

In [32]:
# MSE (Mean Squared Error)

loss = tf.reduce_mean(tf.square(y_true-y_pred))

# Cross Entropy

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
loss = tf.reduce_mean(loss)

### Linear Regression 

- Batch Gradient Descent: entire dataset.
- Stochastic Gradient Descent (SGD): mini-batches. 
- In TensorFlow:

<pre>optimizer = tf.train.GradientDescentOptimizer(learning_rate)</pre>
<pre>train = optimizer.minimize(loss)</pre>


In [33]:
import numpy as np

x_data = np.random.randn(2000,3)
w_real = [0.4, 0.6, 0.2]
b_real = -0.3 
noise = np.random.randn(1,2000)*0.1 
y_data = np.matmul(w_real, x_data.T)+b_real+noise

In [34]:
num_iters = 10 
g = tf.Graph()
wb = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
        b = tf.Variable(0, dtype=tf.float32, name='b')
        y_pred = tf.matmul(w, tf.transpose(x))+b
        
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
        
    with tf.name_scope('training') as scope:
        lr = 0.5
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        for step in range(num_iters):
            sess.run(train, {x:x_data, y_true:y_data})
            if(step%5==0):
                print(step, sess.run([w,b]))
                wb.append(sess.run([w,b]))
                
        print(10, sess.run([w,b]))

0 [array([[ 0.40274513,  0.59352362,  0.17637035]], dtype=float32), -0.31382522]
5 [array([[ 0.40287709,  0.60446119,  0.20664811]], dtype=float32), -0.30317146]
10 [array([[ 0.40287712,  0.60446125,  0.20664825]], dtype=float32), -0.30317137]


### Logistic Regression 

In [35]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

x_data = np.random.randn(20000,3)
w_real = [0.4, 0.6, 0.2]
b_real = -0.3
wb = np.matmul(w_real,x_data.T)+b_real

y_data_bef_noise = sigmoid(wb)
y_data = np.random.binomial(1, y_data_bef_noise)

In [36]:
num_iters = 50
g = tf.Graph()
wb = []

with g.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 3])
    y_true = tf.placeholder(tf.float32, shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]], dtype=tf.float32, name='W')
        b = tf.Variable(0, dtype=tf.float32, name='b')
        y_pred = tf.matmul(w, tf.transpose(x))+b
        
    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)
        loss = tf.reduce_mean(loss)
        
    with tf.name_scope('training') as scope:
        lr = 0.5
        optimizer = tf.train.GradientDescentOptimizer(lr)
        train = optimizer.minimize(loss)
        
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for step in range(num_iters):
            sess.run(train, {x:x_data, y_true:y_data})
            if(step%5==0):
                print(step, sess.run([w,b]))
                wb.append(sess.run([w,b]))
                
        print(50, sess.run([w,b]))

0 [array([[ 0.04112099,  0.06846264,  0.02003187]], dtype=float32), -0.032350261]
5 [array([[ 0.18276456,  0.30256298,  0.08900417]], dtype=float32), -0.14264938]
10 [array([[ 0.26029974,  0.42916155,  0.12672393]], dtype=float32), -0.20180328]
15 [array([[ 0.30538484,  0.50203937,  0.14863394]], dtype=float32), -0.23547475]
20 [array([[ 0.33263034,  0.54571784,  0.16186109]], dtype=float32), -0.25541481]
25 [array([[ 0.34947604,  0.57253945,  0.17003198]], dtype=float32), -0.26751855]
30 [array([[ 0.36003467,  0.58925474,  0.17514937]], dtype=float32), -0.27498147]
35 [array([[ 0.36670724,  0.59976691,  0.1783811 ]], dtype=float32), -0.27963001]
40 [array([[ 0.37094492,  0.60641575,  0.18043235]], dtype=float32), -0.28254512]
45 [array([[ 0.37364432,  0.61063606,  0.18173833]], dtype=float32), -0.28438166]
50 [array([[ 0.37508124,  0.61287636,  0.18243322]], dtype=float32), -0.28535065]


## Softmax Regression with MNIST



<center>![](mnist.png)</center>
*<p style='text-align:right'>By Josef Steppan (Own work), via Wikimedia Commons</p>*

In [37]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [38]:
datadir = '/data'
num_iters = 1000
minibatch_size = 100

In [39]:
data = input_data.read_data_sets(datadir, one_hot=True)
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

correct_pred = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(num_iters):
        batch_xs, batch_ys = data.train.next_batch(minibatch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y_true:batch_ys})
        
    testing = sess.run(accuracy, feed_dict={x:data.test.images, y_true:data.test.labels})
    
print('Accuracy: {:.4}%'.format(testing*100))

Extracting /data\train-images-idx3-ubyte.gz
Extracting /data\train-labels-idx1-ubyte.gz
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz
Accuracy: 91.31%


## Convolutional Neural Networks

<center>![](cnn.png)</center>
*<p style='text-align:right'>By Aphex34 (Own work) [CC BY-SA 4.0], via Wikimedia Commons</p>*

### Good resources:

- CS231n from Stanford University
- Convolutional Neural Networks (Course 4) by Prof. Andrew Ng - Deep Learning Specialization


#### Convolution in TensorFlow:

<center><pre>tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')</pre></center>

- x.shape is [None, 28, 28, 1]
- W.shape is [5, 5, 1, 32]
- strides - how the filter moves across the image
- 'SAME' padding - output size = input size